# [실습] PDF 내용 기반 질의응답 어플리케이션

이번에는 PDF의 내용을 이용해 질의응답을 수행해 보겠습니다.

### 라이브러리 설치  

랭체인 관련 라이브러리와 벡터 데이터베이스 라이브러리를 설치합니다.   

In [ ]:
!pip install openai langchain langchain-openai langchain-community chromadb tiktoken langchain_chroma pymupdf

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('env', override=True)

if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')

OpenAI API 키 확인


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature = 0.7, max_tokens = 4096)

## Indexing : 데이터 불러오기

PyMuPDFLoader를 이용해 목표 문서를 불러옵니다.    
실습 시트에 포함된 교재 PDF 파일을 업로드해 주세요.

In [ ]:
# 현재 위치의 모든 pdf 불러오기
from glob import glob
import os

pdfs = glob(os.path.join('./', '*.pdf'))
pdfs

['./2505.09388v1.pdf']

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

documents = []

for path_material in pdfs:
    print(path_material)
    loader = PyMuPDFLoader(path_material)
    # 페이지별로 저장
    pages = loader.load()
    documents.append(pages)
    print("# Number of documents:", len(pages))

./2505.09388v1.pdf
# Number of documents: 35


PDF Loader로 불러온 데이터는 페이지 단위로 저장됩니다.    

In [ ]:
for i in range(10,15):
    print(pages[i].page_content)
    print('----------')

Table 9: Examples of SFT data for thinking and non-thinking modes during the thinking mode fusion
stage. For the thinking mode, the /think flag can be omitted since it represents the default behavior. This
feature has been implemented in the chat template1 supported by the Hugging Face’s tokenizer, where
the thinking mode can be disabled using an additional parameter enable thinking=False.
Thinking Mode
Non-Thinking Mode
<|im start|>user
{query} /think<|im end|>
<|im start|>assistant
<think>
{thinking content}
</think>
{response}<|im end|>
<|im start|>user
{query} /no think<|im end|>
<|im start|>assistant
<think>
</think>
{response}<|im end|>
stable, which is crucial for maintaining stable training. As a result, we achieve consistent improvements
in both training reward and validation performance over the course of a single RL run, without any
manual intervention on hyperparameters. For instance, the AIME’24 score of the Qwen3-235B-A22B
model increases from 70.1 to 85.1 over a total of

각각의 Document를 하나로 합쳐, 하나의 큰 Document를 만들고 청킹을 수행하겠습니다.

In [ ]:
from langchain_core.documents import Document
# Document 클래스 만들기

corpus = Document(page_content='', metadata={'source':'./교재.pdf'})
for page in pages:
    corpus.page_content += page.page_content + '\n'

corpus.page_content = corpus.page_content.replace('\n\n','\n')
len(corpus.page_content)

116009

Text Splitter를 이용해 청크로 분리합니다.   
이번에는 토큰 단위로 분리해 보겠습니다.    
TextSplitter의 `.from_tiktoken_encoder`를 이용합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=1000,
    chunk_overlap=200,
    # GPT-4o (= GPT 4.1 = GPT 5) 기준 1000/200 토큰 청킹
)

token_chunks = token_splitter.split_documents([corpus])
print(len(token_chunks))

55


전체 데이터를 Chroma에 저장합니다.

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
Chroma().delete_collection()
db = Chroma.from_documents(token_chunks, embeddings)


retriever = db.as_retriever(search_kwargs={"k": 5})
# Top K search 옵션 정하기


# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# Ex) author가 Hyungho Byun인 Document만 검색
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Hyungho Byun'}})

In [ ]:
# Query 검색
unique_docs = retriever.invoke("LangChain의 장점은?")

print(unique_docs)

[Document(id='88c55bbd-0719-45e0-b628-e92a4f4047c5', metadata={'source': './교재.pdf'}, page_content='Qwen2.5-14B-Instruct\n91.4\n97.7\n96.8\n95.9\n93.4\n86.7\n78.1\nQwen2.5-32B-Instruct\n92.9\n96.9\n97.1\n95.5\n95.5\n90.3\n82.0\nQwen2.5-72B-Instruct\n95.1\n97.7\n97.2\n97.7\n96.5\n93.0\n88.4\nNon-thinking\nMode\nQwen3-4B\n85.2\n95.1\n93.6\n91.0\n87.8\n77.8\n66.0\nQwen3-8B\n89.1\n96.3\n96.0\n91.8\n91.2\n82.1\n77.4\nQwen3-14B\n94.6\n98.0\n97.8\n96.4\n96.1\n94.0\n85.1\nQwen3-32B\n93.7\n98.4\n96.0\n96.2\n94.4\n91.8\n85.6\nQwen3-30B-A3B\n91.6\n96.5\n97.0\n95.3\n92.4\n89.1\n79.2\nQwen3-235B-A22B\n95.0\n97.7\n97.2\n96.4\n95.1\n93.3\n90.6\nThinking\nMode\nQwen3-4B\n83.5\n92.7\n88.7\n86.5\n83.2\n83.0\n67.2\nQwen3-8B\n84.4\n94.7\n94.4\n86.1\n80.8\n78.3\n72.0\nQwen3-14B\n90.1\n95.4\n93.6\n89.8\n91.9\n90.6\n79.0\nQwen3-32B\n91.0\n94.7\n93.7\n91.6\n92.5\n90.0\n83.5\nQwen3-30B-A3B\n86.6\n94.1\n92.7\n89.0\n86.6\n82.1\n75.0\nQwen3-235B-A22B\n92.2\n95.1\n94.8\n93.0\n92.3\n92.0\n86.0\nFor evaluating long-

## [실습] PDF 요약 체인 만들기    

Prompt와 Chain을 구성하여, 강의 자료에 대한 질의응답을 수행하는 RAG 체인을 만들고 실행하세요.

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("user", '''당신은 Qwen 3 Technical Report에 대해 QA(Question-Answer)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])

prompt.pretty_print()
# Prompt : 한국어 질문 전달하

================================ Human Message =================================

당신은 Qwen 3 Technical Report에 대해 QA(Question-Answer)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


In [ ]:
from langchain.schema.output_parser import StrOutputParser
translate_prompt = ChatPromptTemplate(
    ["다음 질문을 그대로 영어로 번역하세요: {query}"]
)
translate_chain = translate_prompt | llm | StrOutputParser()

translate_chain.invoke("큐웬 3 컨텍스트 윈도우 어떻게 돼?")

'How many context windows does Qwen 3 have?'

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return " \n---\n ".join(['Content: '+ doc.page_content+ '\n' for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": translate_chain | retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# 테스트

rag_chain.invoke("Qwen 3의 컨텍스트 길이는 어떻게 되니?")


'Qwen3 모델의 컨텍스트 길이는 모델에 따라 다르며, 다음과 같습니다.\n\n- Qwen3-0.6B, Qwen3-1.7B 모델: 32K (32,768 토큰)\n- Qwen3-4B, Qwen3-8B, Qwen3-14B, Qwen3-32B 모델: 128K (131,072 토큰)\n- Qwen3 MoE 모델 (Qwen3-30B-A3B, Qwen3-235B-A22B): 128K (131,072 토큰)\n\n즉, Qwen3 모델들은 최대 128K 토큰의 긴 컨텍스트를 지원합니다.'

Top-K 기반의 RAG도 문서의 내용을 바탕으로 답변하지만,    
특정 문제에 대해서는 단일 검색이 아닌 전체 문서를 확인해야 하는 경우가 존재합니다.   

Chunking을 활용하여, PDF 파일을 요약해 보겠습니다.

## 요약(Summarization)   

### 1. Stuff : 전체 문서를 다 넣고 요약하기

가장 간단한 요약 방법입니다.   
문서의 길이가 Context 길이보다 큰 경우에는 실행이 어렵습니다.

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """ Qwen3의 Technical Report가 아래에 주어집니다.
내용을 요약하여 아래에 출력하세요.
요약은 4개의 문단과 문단별 5개의 문장으로 작성하세요.
---
{text}
---
요약:"""

prompt = PromptTemplate(template = prompt_template)
chain = prompt | llm | StrOutputParser()


answer = chain.invoke({'text':corpus.page_content})
print(answer)

Qwen3는 Qwen 모델 시리즈의 최신 버전으로, 성능, 효율성, 다국어 지원 능력을 크게 향상시킨 대규모 언어 모델(LLM)이다. Qwen3는 0.6억에서 235억 파라미터 규모의 조밀(dense) 및 전문가 혼합(MoE) 아키텍처 모델을 포함하며, 복잡한 다단계 추론을 위한 '생각 모드'와 신속한 응답을 위한 '비생각 모드'를 하나의 통합 프레임워크로 제공한다. 사용자는 작업의 복잡도에 따라 계산 자원을 조절하는 '생각 예산' 기능을 활용할 수 있으며, 이전 모델 대비 29개에서 119개 언어로 다국어 지원 범위를 확대하여 글로벌 활용성을 높였다. 모든 Qwen3 모델은 Apache 2.0 라이선스로 공개되어 연구와 개발의 투명성과 재현성을 보장한다.

Qwen3의 학습에는 36조 개의 토큰 규모의 대규모 다국어 및 다분야 데이터셋이 사용되었으며, PDF 문서에서 텍스트를 추출하는 멀티모달 기법과 수학·코딩 전문 모델을 활용한 합성 데이터 생성으로 데이터 다양성과 품질을 크게 확장하였다. 사전학습은 일반 지식 학습, 추론 능력 강화, 그리고 최대 32,768 토큰의 장문 처리 능력 향상을 위한 3단계로 진행되었고, 모델 구조는 안정적인 학습을 위한 QK-Norm 도입과 고성능을 위한 세부 개선이 포함되었다. 또한, Qwen3는 다양한 벤치마크에서 기존 오픈소스 및 폐쇄형 모델과 경쟁하거나 이를 능가하는 최첨단 성능을 입증하였다.

사후학습(post-training)은 생각 모드와 비생각 모드를 통합하고, 소규모 모델의 효율적 개발을 위한 '강-약 지식 증류(Strong-to-Weak Distillation)'를 중심으로 설계되었다. 네 단계로 구성된 훈련 과정은 긴 사슬 추론(Chain-of-Thought) 학습, 강화 학습을 통한 추론 능력 향상, 모드 통합 및 일반 도메인 강화 학습을 포함한다. 이를 통해 Qwen3는 사용자가 모드를 자유롭게 전환하고, 추론 깊이를 설정할 수 있게 하며, 소형 모델도 고성능을 유지하면서 개발 비용과 시간을 크게 절감할

## 2. map_reduce
map_reduce는 청크 단위로 요약을 만들고, 이를 합친 결과를 이용해 최종 요약문을 다시 작성합니다.   

map 과정과 combine 과정이 존재하며, 각각 다른 프롬프트를 사용할 수 있습니다.

In [ ]:
# Map 과정 : 각 문서에 대해 요약을 생성합니다.
from tqdm import tqdm

map_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
Qwen3 Techinal Report의 내용을 읽고 요약하세요.
요약은 1-3개의 문단과 문단별 5개의 문장으로 작성하세요.
'''),
    ('user', '''{text}
---
Summary:
''')])

raw_summaries = []

map_chain  = map_prompt | llm | StrOutputParser()


for i in tqdm(range(len(token_chunks))):
    response = map_chain.invoke(token_chunks[i].page_content)
    raw_summaries.append(response)
    print('')
    print('#',i)
    print(response)
    print('===========================')

  2%|▏         | 1/55 [00:08<07:50,  8.71s/it]


# 0
Qwen3는 Qwen 모델 시리즈의 최신 버전으로, 성능, 효율성, 다국어 지원을 크게 향상시킨 대형 언어 모델(LLM)들로 구성되어 있다. 이 시리즈는 0.6억에서 2350억 파라미터 규모의 밀집형(dense) 및 전문가 혼합(MoE) 아키텍처 모델을 포함하며, 복잡한 다단계 추론을 위한 '사고 모드'와 빠른 맥락 기반 응답을 위한 '비사고 모드'를 통합하여 하나의 통합된 프레임워크 내에서 동적 전환이 가능하다. 또한, 사용자가 추론 시 계산 자원을 적응적으로 할당할 수 있는 '사고 예산' 메커니즘을 도입해 작업 복잡도에 따라 지연 시간과 성능의 균형을 맞출 수 있다.

Qwen3는 주요 모델의 지식을 활용해 소규모 모델 구축에 필요한 계산 자원을 크게 줄이면서도 경쟁력 있는 성능을 유지하고, 코드 생성, 수학적 추론, 에이전트 작업 등 다양한 벤치마크에서 최첨단 결과를 달성한다. 전작 Qwen2.5와 비교해 지원 언어 수를 29개에서 119개로 대폭 확대하여 다국어 이해 및 생성 능력을 강화함으로써 글로벌 접근성을 향상시켰다. 모든 Qwen3 모델은 Apache 2.0 라이선스 하에 공개되어 연구 및 개발 커뮤니티의 재현성과 협업을 적극 지원한다.

이 보고서는 AGI 목표를 향한 대형 기초 모델의 발전 맥락에서 Qwen3의 설계 철학과 기술적 진보를 설명하며, 대규모 데이터와 강화학습을 활용한 추론 최적화, 오픈소스 생태계에서의 역할 강화 등을 강조한다. 특히, 2350억 파라미터 MoE 모델인 Qwen3-235B-A22B는 높은 성능과 효율적인 추론을 동시에 달성하도록 설계되어 다양한 다운스트림 응용에 적합하다. Qwen3는 사용자 요구에 따라 사고 모드와 비사고 모드를 자유롭게 전환 가능케 하여 다양한 사용 시나리오에 유연하게 대응한다.


  4%|▎         | 2/55 [00:17<07:32,  8.54s/it]


# 1
Qwen3는 다양한 작업과 분야에서 최첨단 성능을 발휘하는 오픈 웨이트 대형 언어 모델(LLM) 모음으로, 0.6억에서 2350억 매개변수까지 다양한 크기의 밀집형 및 Mixture-of-Experts(MoE) 모델을 제공한다. 대표 모델인 Qwen3-235B-A22B는 2350억 매개변수와 토큰당 220억 활성화 매개변수를 갖춘 MoE 구조로, 높은 성능과 효율적인 추론을 동시에 달성한다. 이 모델은 사고 모드와 비사고 모드라는 두 가지 작동 모드를 하나의 모델에 통합하여, 사용자가 작업에 맞게 유연하게 모드 전환을 할 수 있도록 지원한다.

Qwen3는 약 36조 개 토큰, 119개 언어 및 방언을 포함하는 대규모 다국어 데이터로 사전학습되었으며, 멀티모달 접근 방식을 통해 PDF 문서에서 텍스트를 추출하고 수학 및 코딩 관련 합성 데이터를 생성하는 등 훈련 데이터를 확장했다. 사전학습은 일반 지식을 쌓는 1단계, STEM 및 코딩 관련 추론 능력을 강화하는 2단계, 그리고 최대 컨텍스트 길이를 4,096에서 32,768 토큰으로 확장하는 3단계로 이루어진다. 이러한 단계적 학습 전략은 모델의 전반적 성능과 다중 언어 처리 능력을 크게 향상시킨다.

사후학습에서는 장기 체인오브생각(CoT) 및 강화학습을 활용해 추론 능력을 극대화하고, 사고 및 비사고 모드를 모두 다룰 수 있도록 데이터 통합 및 일반 도메인 강화학습을 수행한다. 소형 모델들은 대형 모델로부터 지식을 전이하는 강한-약한(distillation) 방식을 통해 효율적으로 성능을 높였다. 평가 결과, Qwen3의 사전학습 및 사후학습 모델 모두 다양한 벤치마크에서 최첨단 성능을 기록했으며, 특히 코딩, 수학, 에이전트 관련 작업에서 뛰어난 성과를 보였다.


  5%|▌         | 3/55 [00:24<06:47,  7.84s/it]

...


# 52
Qwen3 기술 보고서는 다국어 이해 및 수학적 추론 능력을 포함한 최신 인공지능 언어 모델의 성능 평가와 개선 방향을 종합적으로 다룬다. 보고서는 다양한 다국어 벤치마크와 체인 오브 사고(chain-of-thought) 추론 기법을 통해 모델의 복잡한 문제 해결 능력을 검증하며, 희귀 단어 번역을 위한 서브워드 단위 활용과 회전 위치 임베딩(rotary position embedding) 등의 기술적 혁신을 소개한다. 또한, 수학적 추론 능력을 다국어 환경에서 평가하는 PolyMath와 DeepSeekMath 같은 첨단 평가 도구를 활용해 모델의 한계와 가능성을 심층 분석한다.

보고서에서 강조하는 주요 이슈는 모델의 일반화 능력과 지역별 지식의 통합, 그리고 테스트 시 스케일링(test-time scaling)의 언어적 일반화 가능성이다. 이러한 연구들은 빅벤치(BIG-Bench)와 MMLU-Pro 같은 강력하고 도전적인 다중 작업 언어 이해 벤치마크를 통해 입증되며, 모델이 다국어 및 수학 문제 해결에서 보여주는 성능 향상을 목표로 한다. 또한, LiveBench와 WritingBench 같은 새로운 벤치마크는 오염 없는 데이터 환경에서 생성형 쓰기 능력을 포함한 다양한 언어 생성 태스크를 평가하는 데 기여한다.

종합적으로 Qwen3 보고서는 인공지능 언어 모델이 다언어 환경과 수학적 복잡성을 효과적으로 다룰 수 있도록 설계되고 평가되는 최신 연구 동향을 반영한다. 이는 자연어 처리, 기계 번역, 수학적 추론, 그리고 생성형 AI 분야에서의 기술 발전과 실용적 적용 가능성을 높이는 데 중요한 역할을 한다. 앞으로도 이와 같은 다각도의 평가와 혁신적 기법 도입이 모델의 정확도와 범용성을 강화하는 데 필수적임을 시사한다.


 98%|█████████▊| 54/55 [06:07<00:06,  6.38s/it]


# 53
Qwen3 기술 보고서는 대규모 언어 모델(LLM)의 최신 발전과 성능 향상에 관한 내용을 다루고 있다. 보고서는 다양한 벤치마크와 비교 평가를 통해 Qwen 시리즈 모델의 우수성을 입증하며, 특히 Qwen2, Qwen2.5, 그리고 Qwen2.5-Math 모델들의 기술적 세부사항과 성능 개선 전략을 상세히 설명한다. 이들 모델은 데이터 혼합 최적화, 장기 문맥 처리 능력 향상, 그리고 수학적 문제 해결 능력 강화에 중점을 두고 개발되었다고 밝히고 있다.

또한, 보고서는 LiveBench, WritingBench, Berkeley function calling leaderboard 등 최신 LLM 평가 벤치마크와의 연계 연구를 통해 모델의 실제 적용 가능성과 신뢰성을 높이고자 하는 시도를 소개한다. 특히 오염 없는 데이터 셋을 활용한 엄격한 평가와 생성형 글쓰기 능력 검증이 중요한 평가 지표로 다뤄진다. 이와 같이 Qwen3는 다면적 평가를 기반으로 한 실용적이고 확장성 있는 LLM 개발을 목표로 한다.

마지막으로, 보고서는 향후 연구 방향으로 추론 에이전트 시대에 적합한 모델 설계, 데이터 전처리 및 혼합 전략의 지속적 최적화, 그리고 전문 영역(예: 수학) 모델의 자기 개선 메커니즘 개발을 제시한다. 이를 통해 Qwen 시리즈는 인공지능 언어 모델 분야에서 경쟁력을 유지하며, 다양한 응용 분야에서의 활용 가능성을 넓히려는 비전을 공유한다.


100%|██████████| 55/55 [06:16<00:00,  6.84s/it]


# 54
Qwen3는 최신 대규모 언어 모델(LLM)로, Qwen2.5 시리즈의 발전을 기반으로 설계되었다. 이 모델은 텍스트와 코드, 멀티모달 데이터를 통합하여 다양한 자연어 처리 및 생성 작업에서 뛰어난 성능을 보인다. 특히, 수학적 추론 능력 강화를 위해 Qwen2.5-Math에서 도입된 자기 개선(self-improvement) 기법을 확장 적용하였으며, 이를 통해 전문적인 수학 모델로서의 역량을 크게 향상시켰다. 또한, 다국어 및 다중 작업 평가를 위해 P-MMEval 벤치마크를 활용해 다양한 언어와 작업 간의 일관된 성능 평가를 수행하였다. 이와 함께, Instruction-following 및 논리 추론 능력 평가를 위한 AutoLogi 등 자동화된 평가 도구도 병행하여 개발, 모델의 실용성과 신뢰성을 높였다.

Qwen3 기술 보고서는 학습 데이터, 모델 아키텍처, 훈련 방법론, 그리고 평가 결과를 상세히 다룬다. 모델은 대규모 멀티태스크 학습과 고도화된 지시어 이해 능력을 바탕으로 복잡한 지시를 정확히 수행하며, 다양한 도메인에 걸쳐 강력한 일반화 능력을 보인다. 특히, 수학 및 논리 퍼즐과 같은 고난도 문제에서 기존 모델 대비 우수한 성능을 기록하였으며, 이는 모델의 자기 개선 및 멀티모달 학습 전략이 실질적 효과를 발휘했음을 의미한다. 평가 결과는 Qwen3가 현재 공개된 주요 LLM들과 비교해도 경쟁력 있는 수준임을 입증한다. 아울러, 지속적인 연구를 통해 모델의 확장성과 효율성 향상 방안도 제시하고 있다.

종합적으로, Qwen3는 대규모 언어 모델 연구에 있어 자기 개선, 멀티태스크 및 멀티모달 학습, 그리고 일관된 평가 체계 구축 등 다각도의 혁신을 포함한다. 이는 자연어 이해, 생성, 수학적 추론, 그리고 다국어 처리 능력에서 새로운 기준을 제시하며, 차세대 인공지능 응용에 중요한 기반 기술로 자리매김하고 있다. 앞으로도 Qwen 시리즈의 발전을 통해 더욱 정교하고 신뢰성 높은 AI 모델 개발이 기대된다.


In [ ]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
Qwen3 Techinal Report에 대한 요약문의 리스트가 주어집니다.
이를 읽고, 전체 주제를 포함하는 최종 요약을 작성하세요.
요약은 5개의 문단과 문단별 4-8개의 문장으로 작성하세요.
'''),
    ('user', '''{text}
---
Summary:
''')])

reduce_chain = reduce_prompt | llm | StrOutputParser()

summary = reduce_chain.invoke('\n---\n'.join(raw_summaries))

summary

'Qwen3는 Qwen 시리즈의 최신 대형 언어 모델(LLM)로, 0.6억에서 2350억 파라미터 규모의 밀집형(Dense)과 전문가 혼합형(MoE) 모델을 포함한다. 이 모델은 ‘사고 모드’와 ‘비사고 모드’를 하나의 프레임워크에서 동적으로 전환할 수 있으며, 계산 자원을 작업 복잡도에 맞게 조절하는 ‘사고 예산’ 메커니즘을 도입해 성능과 지연 시간의 균형을 최적화한다. 119개 언어와 방언, 36조 토큰에 달하는 대규모 다국어 데이터를 활용해 사전학습되었으며, 단계별 학습 전략과 강화학습, 증류 기법을 통해 추론 능력과 다국어 이해력을 크게 향상시켰다. Apache 2.0 라이선스로 공개되어 연구 및 산업계에서 협업과 재현성을 지원한다.\n\n아키텍처 측면에서 Qwen3는 최신 Transformer 기법을 적용하고, MoE 모델은 128명의 전문가를 활용해 활성화된 파라미터 수를 줄이면서도 고성능을 구현한다. 대표 모델인 Qwen3-235B-A22B는 2350억 파라미터 중 220억 활성화 파라미터를 사용해 수학, 코딩, 다국어 등 다양한 분야에서 최첨단 성능을 보인다. 토크나이저는 바이트 수준의 Byte-Pair Encoding을 기반으로 약 15만 개 어휘를 포함하며, 최대 128K 토큰의 긴 문맥도 처리 가능해 대규모 문서 이해에 강점을 가진다.\n\n사후학습에서는 사고 모드와 비사고 모드의 융합(Thinking Mode Fusion)과 강한 모델에서 약한 모델로 지식을 전이하는 강한-약한 증류(Strong-to-Weak Distillation) 파이프라인을 도입했다. 이를 통해 경량 모델도 고성능과 사고 모드 전환 능력을 갖추면서 훈련 비용과 시간을 크게 줄였다. 강화학습과 감독학습이 결합된 학습 전략은 다양한 작업에서 일관된 성능 향상을 가져왔으며, 사고 예산 조절 기능은 복잡한 문제에서 효율적이고 유연한 추론을 가능하게 한다.\n\n성능 평가에서는 Qwen3 시리즈가 다양한 일반, 수학, 코딩, 에이전트, 다국어, 정렬(Alignment) 벤치마

## 3. Refine
Refine은 청크를 순서대로 참고하며, 매 시점 요약문을 작성합니다.   
요약문과 새로운 청크를 비교하여, 요약문을 업데이트합니다.



In [ ]:
# 첫 번째 청크 요약하기

first_summarize_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
Qwen3 Techinal Report의 내용을 읽고 요약하세요.
요약은 1-3개의 문단과 문단별 5개의 문장으로 작성하세요.
'''),
    ('user', '''{text}
---
Summary:
''')])

X = first_summarize_prompt.format_messages(text=token_chunks[0].page_content)

summary = llm.invoke(X).content
summary

In [ ]:
# 두 번째 청크부터, 계속 추가하며 요약하기
refine_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
Qwen3 Techinal Report의 현재 시점까지의 요약이 주어집니다.
이를 읽고, 새롭게 주어지는 내용과 비교하여 요약을 보완하세요.
요약은 5개의 문단과 문단별 4-8개의 문장으로 작성하세요.
'''),
    ('user', '''현재 시점까지의 요약: {previous_summary}
---
새로운 내용: {new_text}''')])


refine_chain = refine_prompt | llm | StrOutputParser()

for i in tqdm(range(1, len(token_chunks))):
    summary = refine_chain.invoke(
        {'previous_summary':summary,
         'new_text':token_chunks[i].page_content})
    print('')
    print(summary)
    print('=======================')

# 길이를 지정하지 않으면 오래 걸릴 수 있습니다.

In [ ]:
summary

## [실습] 임의의 PDF 다운로드하여 요약하기

arxiv 등의 페이지에서 PDF를 다운로드하여 업로드하고,   
Stuff/Map-Reduce/Refine 등의 방법을 이용해 전체 PDF를 요약하세요.